In [47]:
from Bio import SearchIO
from Bio.SeqIO import *

from tst.domutil.util import *
from tst.utils_db import *


# os.environ["SEQlib"] = full("$DATA/sequence-data")

In [53]:
import sys
reload(sys.modules["tst.domutil.util"])
from tst.domutil.util import *

In [ ]:
# reload(tst.domutil.util)
%load_ext autoreload
%autoreload 0

In [ ]:
### Manually generated tmp.hmmpf and tmp.domtbl

### search_
wkdir = "/data/sequence-data/"
hmm_profile = "%s/tmp.hmmpf" % wkdir
hmm_domtbl = "%s/tmp.domtbl" % wkdir
# CATH_S35

# kwargs = {"format":"hmmer3-domtab"}
tbl_format = "hmmsearch3-domtab"

In [ ]:
import os
os.environ["SEQlib"]

## import sequence database

In [ ]:
print seqDB_curr.sequence_set.count()
print sequence.objects.count()

In [13]:
jdict = {
    'name':'CATH',
    'version':'4_1_0'}
seqDB_curr = verify_exist_entry(jdict,seqDB)

INPUT = full("$SEQlib/cath-domain-seqs-S100.fa")
fmt = 'fasta'
p_cathFAheader=re.compile('\|([0-9,a-z,A-Z]*)\/')
f_header = lambda s:p_cathFAheader.findall(s.id)[0]


In [ ]:
jdict = {
    'name':'CATH-S40-nr',
    'version':'4_1_0'}
seqDB_curr = verify_exist_entry(jdict,seqDB)

INPUT = full("$SEQlib/cath-dataset-nonredundant-S40.fa")
fmt = 'fasta'
p_cathFAheader=re.compile('\|([0-9,a-z,A-Z]*)\/')
header2acc = lambda s:p_cathFAheader.findall(s.id)[0]


In [15]:
jdict = {
    'name':'Pfam-A',
    'version':'rp15'}
seqDB_curr = verify_exist_entry(jdict,seqDB)


INPUT = full("$SEQlib/Pfam-A_rp15")
fmt = 'stockholm'
header2acc = lambda s: s.name


In [21]:
%%time
behave = "import sequence database"

if "debug" not in locals(): debug = 0
reset_database_connection()
from Bio import SeqIO

p_clean = re.compile('[^-,^\.]')
oldseqs = seqDB_curr.sequence_set.all()


from time import time
t0 = time()
useFisrt = 1
failcount = 0

iterator = SeqIO.parse( INPUT, fmt )
c = counter(iterator,INF=1,per = 1000)
iterator = SeqIO.parse( INPUT, fmt )

with transaction.atomic():
    for obj in iterator:

        acc = header2acc( obj )
        seq = p_clean.sub('',
                          obj._seq.tostring())
        try:
            if acc.islower():
                raise Exception('accession is all lowercase!')
            jdict = {
                'acc':acc,
                'length':length,
                'seqDB':seqDB_curr,
            }

            seqobjs = oldseqs.filter(**jdict)
            if not seqobjs.exists():
                seqobj = sequence(**jdict)
                seqobj.save()
#                     print "created %s"%jdict['acc']
            elif seqobjs.count() > 1:                   
                print jdict['acc']
            else:
                pass
            
        except Exception as e:
            msg = "%s failed for %s " % (acc, e)
            if debug: print >>sys.__stdout__, msg
            c.fail( msg , acc)
        finally:
            c.count()
        

c.summary( behave,INPUT )

# print "finshed %s from %s" % (behave, INPUT)
# print '%d instances of %d failed' % ( c.f, c.i)
# print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
# failrate = c.f/float(c.i) 
# assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % (failrate*100)

0 of -1
1000 of -1
2000 of -1
3000 of -1
4000 of -1
5000 of -1
6000 of -1
7000 of -1
8000 of -1
9000 of -1
10000 of -1
11000 of -1
12000 of -1
13000 of -1
14000 of -1
15000 of -1
16000 of -1
17000 of -1
18000 of -1
19000 of -1
20000 of -1
21000 of -1
22000 of -1
23000 of -1
24000 of -1
25000 of -1
26000 of -1
27000 of -1
28000 of -1
29000 of -1
30000 of -1
31000 of -1
32000 of -1
33000 of -1
34000 of -1
35000 of -1
36000 of -1
37000 of -1
38000 of -1
39000 of -1
40000 of -1
41000 of -1
42000 of -1
43000 of -1
44000 of -1
45000 of -1
46000 of -1
47000 of -1
48000 of -1
49000 of -1
50000 of -1
51000 of -1
52000 of -1
53000 of -1
54000 of -1
55000 of -1
56000 of -1
57000 of -1
58000 of -1
59000 of -1
60000 of -1
61000 of -1
62000 of -1
63000 of -1
64000 of -1
65000 of -1
66000 of -1
67000 of -1
68000 of -1
69000 of -1
70000 of -1
71000 of -1
72000 of -1
73000 of -1
74000 of -1
75000 of -1
76000 of -1
77000 of -1
78000 of -1
79000 of -1
80000 of -1
81000 of -1
82000 of -1
83000 of -1
84000

KeyboardInterrupt: 

## Importing HMM profiles from .lib file

In [ ]:
%load_ext autoreload
%autoreload 2


In [2]:
header2acc = lambda s:p_cathFAheader.findall(s)[0]
INPUT = full("$SEQlib/cath-S35-hmm3.lib")


In [28]:
'1pkma00'.islower()

True

In [ ]:
seqheader_parse_cath(p_header.findall(hmm_text)[0]).get("acc")

In [7]:
behave = "importing HMM profiles"

from time import time
from tst.domutil.util import *
import re

p_header = re.compile("NAME.*?\n")

lst = parse_hmmlib( INPUT )
c = counter( lst,INF = 1, per = 1000 )
lst = parse_hmmlib( INPUT )
failcount = 0

try:
    with transaction.atomic():
        for hmm_text in lst:
            c.count()
            header = p_header.findall(hmm_text)[0]
            acc = header2acc(header)
            length = int(p_hmmlen.findall(hmm_text)[0])
            
#             acc = next(header_gen)
            try:
                cnode = domain.objects.get(domain_id = acc).classification
                if cnode.hmmprofile_set.exists():
                    pass
                else:
                    hmm = HMMprofile(
                                cath_node_id = cnode.id,
                                text = hmm_text,
                                length = length
                                )
                    hmm.save()
            except Exception as e:
                msg = "failed for %s for Exception:%s" %( acc, e)
                c.fail( msg, acc )
                
except Exception as e:
    print "early stop due to %s" % e


c.summary( behave,INPUT )

# print '%d instances of %d failed' % (c.f, c.i)
# print 'Ended after %.4f sec' % ( time() - t0 )   # len(lst)d
# failrate = c.f / float(c.i) 
# assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate

0 of -1
failed for 1f0cA01 for Exception:get() returned more than one domain -- it returned 2!
1000 of -1
failed for 1imvA01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1imvA02 for Exception:get() returned more than one domain -- it returned 2!
2000 of -1
failed for 1jjoC01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1jmoA01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1jmoA02 for Exception:get() returned more than one domain -- it returned 2!
failed for 1k99A00 for Exception:get() returned more than one domain -- it returned 2!
failed for 1k9oI01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1k9oI02 for Exception:get() returned more than one domain -- it returned 2!
failed for 1l8yA00 for Exception:get() returned more than one domain -- it returned 2!
failed for 1lj5A01 for Exception:get() returned more than one domain -- it returned 2!
failed for 1m

## HMMsearch all the loaded HMM profiles (against CATHS100)

In [ ]:
%load_ext autoreload
%autoreload 0

In [ ]:
from time import time
from tst.domutil.util import *
import re
import gc
# p_header = re.compile("NAME.*?\n")
# from copy import copy
# hmmlib_file = full("$SEQlib/cath-S35-hmm3.lib")
# header_gen = (seqheader_parse_cath(line.rstrip("\n")[6:])["acc"] for  index_hmmlib(hmmlib_file))
seqDB_curr = seqDB.objects.get(name = 'CATH')
seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"
behave = "HMMsearching profiles"

HMMprofile.objects.all()

t0 = time()

qset = HMMprofile.objects.order_by("id")
c = counter(range(qset.count()), per = 100)
batches = batch_qs(qset, batch_size =500)
# it = lst.iterator()
failcount = 0

# print "hi"
def worker():
    q_hits = hmmsearch(hmm,
    seqDB_curr = seqDB_curr,
    seqDB_file = seqDB_file,
    tmpdir = "/tmp/hmmsearch"                
    )

    oldhits = hmm.hit4hmm2hsp_set.all()
    for hit in q_hits:
        hsp = hit[0] ### Assume only one dom per hit
        jdict = hsp2jdict(hsp, query = hmm )
        jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"] ).id

        if not oldhits.filter(**jdict).exists():
            hit_db = hit4hmm2hsp(**jdict)
            hit_db.save()

transaction.set_autocommit( False)
if 1:
# with transaction.atomic():
    for batch in batches:
        for hmm in batch:
            c.count()
#             if c.i > 500:
#                 continue
            try:
                worker()
#                 hits = hmmsearch(hmm,
#                 seqDB_curr = seqDB_curr,
#                 seqDB_file = seqDB_file,
#                 )
            except Exception as e:
                c.fail("failed for %s for %s" % (hmm,e),)
#                 failcount += 1
            if not c.i % c.per:
                transaction.commit()



print "finshed %s from %s" % (behave, hmmlib_file)
print '%d instances of %d failed' % (c.f,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = c.f/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % failrate
transaction.set_autocommit(True)


## Load HMM search results from flatfile (against CATHS100)

In [37]:
from tst.domutil.util import *
from Bio import SearchIO
INPUT = full('$SEQlib/hmm/CATHS100.domtbl')
INPUT = full('$SEQlib/hmm/CATHS100_v410.domtbl')
f_handles  = split_file(INPUT,number = 6 )  
parser = SearchIO.parse( INPUT, 'hmmsearch3-domtab' )

seqDB_curr = seqDB.objects.get(name = 'CATH')
seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"

c0 = counter([],INF=1,per = 2)

            
acc2hmm_raw  = dict(HMMprofile.objects.defer('text').values_list("cath_node__domain__domain_id","id"))
acc2seq_raw = dict(seqDB_curr.sequence_set.filter().values_list('acc','id'))



In [41]:
__name__='__main__'
__name__

'__main__'

In [17]:
__name__='__test__'
__name__

'__test__'

In [48]:

def parse_worker(fakefile, fmt = 'hmmsearch3-domtab'):
#     print >> sys.__stdout__,fakefile.closed
    parser = SearchIO.parse(fakefile, fmt)
    for q_hits in parser:
    #     print q_hits
        acc = p_cathdomain.findall(q_hits.id)[0]

        try:
            c1 = counter([],INF=-1,ifprint = 0 )
            query_id = acc2hmm[ acc ]
            Jdict = {'query_id':query_id,
                    'hits' : [], }
            for hit in q_hits:
                hsp = hit[0]
                target_acc = p_cathdomain.findall(hit.id)[0]
                try:
                    jdict = hsp2jdict( hsp,simple = 1)
                    jdict['query_id'] = query_id
                    jdict['target_id'] = acc2seq[target_acc]
                    Jdict['hits'].append(jdict)
                except BaseException as e:
                    c1.fail('', target_acc)
                finally:
                    c1.count()

            if c1.f:
                c0.fail('%d of %d instances failed for HMM %s \n Most Recent Exception' % (c1.f, c1.f, acc, c1.e ), acc)
            q.put( Jdict )


        except BaseException as e:
            try:
                c0.fail('%s failed for %s'%(q_hits.id, e), q_hits.id)
            except:
                pass
        finally:
            c0.count()

            
            
def db_listener():
    c = counter([],INF=1, stdout = sys.__stdout__)
    with transaction.atomic():
        while 1:
            
            try:
                obj = q.get()
                c.count()
                if not obj:
                    break
                else:
                    hmm = HMMprofile.objects.defer('text').get(id = obj['query_id'])
                    oldhits = list(
                        hmm.hits.values_list('id',flat = True)
                    )
                    bulk = []

                    for jdict in obj['hits']:
                        if jdict['target_id'] in oldhits:
                            continue
                        else:
                            db_hit = hit4hmm2hsp(**jdict)
                            bulk.append( db_hit )
#                             db_hit.save()
                    hit4hmm2hsp.objects.bulk_create( bulk )
                    
            except BaseException as e:
                print e
                c.fail('%s'%e,)
#                 break
    return c
# tpf=TemporaryFile('w+')


In [56]:
# f_handles = [f0]
ts = []
# for pcount in [9,10,11,12]:
pcount = 9


INPUT = full('$SEQlib/hmm/CATHS100_v410.domtbl')
# INPUT = split_file(  INPUT ,linecount = 10000)[0]
INPUT = split_file(  INPUT ,linecount = 100000)[0]


import sys
if 1:
    f_handles = split_file( INPUT,number = pcount - 2)
#     f_handles = split_file( f0,number = pcount - 2)


    import os
    import multiprocessing as mp
    # from 
    from multiprocessing.managers import BaseManager, SyncManager
    # class MyManager(BaseManager): pass
    class MyManager(SyncManager): pass
    MyManager.register('counter',counter)
    from time import time

    t0 = time()
    if __name__=='__main__':
    #     m = mp.Manager()
        m = MyManager()
        m.start()
    #     raise Exception
        c0 = m.counter([],INF=1, ifprint = 1)
        acc2hmm = m.dict(acc2hmm_raw)
        acc2seq = m.dict(acc2seq_raw)

        q = m.Queue();          

        if 1:
            pool = mp.Pool( pcount )
    #     with mp.Pool(mp.cpu_count() - 1) as pool:
            watcher  = pool.apply_async(db_listener, args = [])
            jobs = []
            for f_handle in f_handles:
                print f_handle

                job  = pool.apply_async(parse_worker, args = (f_handle,))
                jobs.append(job)

            for job in jobs:
                job.get()
            q.put(None)
            watcher.get()
            
            
        ###### Delete temporary files
        for f in f_handles:
            os.remove(f)


    t = time() - t0
    ts.append(t)
    print pcount, t 
    
    # print time() - t0

0 of -1
/tmp/feng0s_2qV/0
/tmp/feng0s_2qV/1
/tmp/feng0s_2qV/2
/tmp/feng0s_2qV/3
/tmp/feng0s_2qV/4
/tmp/feng0s_2qV/5
/tmp/feng0s_2qV/6
100 of -1
200 of -1
300 of -1
 300 of -1
400 of -1
 400 of -1
500 of -1
9 19.0308241844


In [46]:
####Manual removal of temp files
for f in f_handles:
            os.remove(f)